In [1]:
import pprint
import pandas as pd

from tastytrade import Credentials
from tastytrade.session import (
    SessionHandler,
)
from tastytrade.intruments import request_options_chains

from injector import Injector

TEST = True
ENV = "Live"

In [ ]:
def configure(binder):
    binder.bind(Credentials, to=Credentials(env=ENV))


injector = Injector([configure])
session = injector.get(SessionHandler)

# session.create_session()
session.get_api_quote_token()

In [ ]:
print("\n", pprint.pformat(vars(session)))

if TEST:
    session.create_session(**vars(Credentials(env=ENV)))

In [ ]:
new_response = request_options_chains(session, "SPX")

In [ ]:
options_chains_df = pd.DataFrame(new_response.json()["data"]["items"])

is_tradeble = options_chains_df["active"] & ~options_chains_df["is-closing-only"]
options_chains_df.loc[is_tradeble].head(3)

In [ ]:
columns = [
    "underlying-symbol",
    "option-type",
    "strike-price",
    "streamer-symbol",
    "symbol",
]

rows = (
    (options_chains_df["settlement-type"] == options_chains_df["settlement-type"].max())
    & (options_chains_df["days-to-expiration"] == options_chains_df["days-to-expiration"].min())
    & (options_chains_df["active"] == 1)
    & ~options_chains_df["is-closing-only"]
)

options_chains_df.loc[rows, columns].sample(5)

In [ ]:
options_chains_df.loc[rows, columns]

In [ ]:
session.close_session()

# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## TastyTrade API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)

## Implementation Notes
While previous experience has been primarily with traditional threading, this project will explore asyncio-based implementation for its potential benefits in handling WebSocket connections.

In [9]:
import threading, time, websocket

# Based off WebSocket example found here: https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py